# Centralized Solution of Multi-Regional Transmission Expansion Horizontal Investment Coordination Problem

### This notebook contains codes for the three zone test case (consisting of the IEEE 14 nodes, IEEE 30 nodes, and a 5 node test system for each of the zones) under the assumption that a single entity solves the problem centrally

#### The first code block loads the different package dependencies to create the virtual environment in Julia

In [1]:
# activate julia environment in a controlled way
julia_environment = "../central_control_PyJuMP/Julia_src/activate.jl"
include(julia_environment)

 Activating new environment at `~/code/Horizontal_Proper/Example/CentJulEnv/Project.toml`
   Updating registry at `~/.julia/registries/General`
######################################################################### 100.0%
  Resolving package versions...
  Installed MozillaCACerts_jll ─ v2022.10.11+0
  Installed libsodium_jll ────── v1.0.19+0
Updating `~/code/Horizontal_Proper/Example/CentJulEnv/Project.toml`
  [7073ff75] + IJulia v1.23.3
Updating `~/code/Horizontal_Proper/Example/CentJulEnv/Manifest.toml`
  [0dad84c5] + ArgTools v1.1.1
  [56f22d72] + Artifacts v1.3.0
  [8f4d0f93] + Conda v1.7.0
  [f43a241f] + Downloads v1.6.0
  [7073ff75] + IJulia v1.23.3
  [692b3bcd] + JLLWrappers v1.4.1
  [682c06a0] + JSON v0.21.3
  [b27032c2] + LibCURL v0.6.3
  [deac9b47] + LibCURL_jll v7.70.0+2
  [29816b5a] + LibSSH2_jll v1.9.0+3
  [739be429] + MbedTLS v1.0.3
  [c8ffd9c3] + MbedTLS_jll v2.16.8+1
  [14a3606d] + MozillaCACerts_jll v2022.10.11+0
  [ca575930] + NetworkOptions v1.2.0
  [69de0a69] + P

Activating the Julia virtual environment


Updating `~/code/Horizontal_Proper/Example/CentJulEnv/Project.toml`
  [82193955] + SCIP v0.10.0
Updating `~/code/Horizontal_Proper/Example/CentJulEnv/Manifest.toml`
  [82193955] + SCIP v0.10.0
  [e5ac4fe4] + SCIP_jll v0.1.3+0
  [508c9074] + bliss_jll v0.73.0+1


Status `~/code/Horizontal_Proper/Example/CentJulEnv/Project.toml`
  [6e4b80f9] BenchmarkTools v1.3.2
  [336ed68f] CSV v0.8.2
  [9961bab8] Cbc v0.9.1
  [e2554f3b] Clp v0.9.1
  [a93c6f00] DataFrames v0.22.7
  [864edb3b] DataStructures v0.18.13
  [60bf3e95] GLPK v0.15.3
  [87dc4568] HiGHS v0.3.0
  [7073ff75] IJulia v1.23.3
  [b6b21f68] Ipopt v0.9.1
  [4076af6c] JuMP v0.22.3
  [b8f27783] MathOptInterface v0.10.9
  [fdba3010] MathProgBase v0.7.8
  [91a5bcdd] Plots v1.15.2
  [731186ca] RecursiveArrayTools v2.20.0
  [82193955] SCIP v0.10.0
  [2913bbd2] StatsBase v0.33.21
  [fdbf4ff8] XLSX v0.8.4
  [ddb6d928] YAML v0.4.8
  [ade2ca70] Dates
  [37e2e46d] LinearAlgebra
  [10745b16] Statistics


In [2]:
using XLSX
using CSV
using GLPK
using DataFrames
using JuMP

┌ Info: Precompiling XLSX [fdbf4ff8-1666-58a4-91e7-1b58723a45e0]
└ @ Base loading.jl:1278
┌ Info: Precompiling CSV [336ed68f-0bac-5ca0-87d4-7b16caf5d00b]
└ @ Base loading.jl:1278
┌ Info: Precompiling GLPK [60bf3e95-4087-53dc-ae20-288a0d20c6a6]
└ @ Base loading.jl:1278
┌ Info: Precompiling DataFrames [a93c6f00-e57d-5684-b7b6-d8193f3e46c0]
└ @ Base loading.jl:1278
┌ Info: Precompiling JuMP [4076af6c-e467-56ae-b986-b466b2749572]
└ @ Base loading.jl:1278


In [5]:
Pkg.status()


Status `~/code/Horizontal_Proper/Example/CentJulEnv/Project.toml`
  [6e4b80f9] BenchmarkTools v1.3.2
  [336ed68f] CSV v0.8.2
  [9961bab8] Cbc v0.9.1
  [e2554f3b] Clp v0.9.1
  [a93c6f00] DataFrames v0.22.7
  [864edb3b] DataStructures v0.18.13
  [60bf3e95] GLPK v0.15.3
  [87dc4568] HiGHS v0.3.0
  [7073ff75] IJulia v1.23.3
  [b6b21f68] Ipopt v0.9.1
  [4076af6c] JuMP v0.22.3
  [b8f27783] MathOptInterface v0.10.9
  [fdba3010] MathProgBase v0.7.8
  [91a5bcdd] Plots v1.15.2
  [731186ca] RecursiveArrayTools v2.20.0
  [82193955] SCIP v0.10.0
  [2913bbd2] StatsBase v0.33.21
  [fdbf4ff8] XLSX v0.8.4
  [ddb6d928] YAML v0.4.8
  [ade2ca70] Dates
  [37e2e46d] LinearAlgebra
  [10745b16] Statistics


#### The next code block reads the different files corresponding to shared candidate lines, internal candidate lines, shared existing lines, internal existing lines, generators, and load

In [3]:
shared_cand = DataFrame(XLSX.readtable("../Input_Data/CandLine.csv", "Taul1", header=true)) #Dataframe of shared candidate lines
int_cand =  DataFrame(XLSX.readtable("../Input_Data/CandLineInt.csv", "Taul1", header=true)) #Dataframe of internal candidate lines
shared_ex =  DataFrame(XLSX.readtable("../Input_Data/SharedEline.csv", "Taul1", header=true)) #Dataframe of shared existing lines
int_ex = DataFrame(XLSX.readtable("../Input_Data/Tran.csv", "Taul1", header=true)) #Dataframe of internal existing lines
gen =  DataFrame(XLSX.readtable("../Input_Data/Gen.csv" , "Taul1", header=true)) #Dataframe of generators
load =  DataFrame(XLSX.readtable("../Input_Data/Load.csv", "Taul1", header=true)) #Dataframe of loads
scen_prob = DataFrame(CSV.File("../Input_Data/Scenario_Probability.csv", header=true)) #Dataframe of scenario probabilities
zone_summary = DataFrame(CSV.File("../Input_Data/Zone_Summary.csv", header=true)) #Dataframe of region-number of nodes

,Zone,Nodes_Total
,Int64,Int64
1,1,14
2,2,30
3,3,5


#### The following code-section constructs several DataFrames for each zone or region, for loads within a particular zone, generators within a particular zone, shared candidate lines originating from or terminating within a particular zone, internal candidate lines within a particular zone, shared existing lines originating from or terminating within a particular zone, internal existing lines within a particular zone, marginal costs of generators within a particular zone, and two binary vectors for each zone, one for checking which all shared candidate lines belong to a particular zone and the other for checking which all shared existing lines belong to a particular zone.

In [4]:
l(i,s) = load[load.zoneNum .== i, [1,2,s+2]] # load within zone i
g(i) = gen[gen.zoneNum .== i, :]   # generators within zone i
shared_c(i) = vcat(shared_cand[shared_cand.nodeZone1 .== i,:] , shared_cand[shared_cand.nodeZone2 .== i, :]) #shared candidate lines within zone i
int_c(i) = int_cand[int_cand.zoneNum .== i, :]   # number of internal candidate lines within zone i
shared_e(i) = vcat(shared_ex[shared_ex.nodeZone1 .== i,:] , shared_ex[shared_ex.nodeZone2 .== i, :]) #shared existing lines within zone i
int_e(i) =int_ex[int_ex.zoneNum .== i, :]       # internal existing lines within zone i
MC(i) = (g(i).C2 .* (g(i).PgMax .^ 2) .+ g(i).C1 .* g(i).PgMax .- g(i).C2 .*(g(i).PgMin .^ 2) .- g(i).C1.* g(i).PgMin) ./ (g(i).PgMax .- g(i).PgMin) #Marginal cost of generators within zone i
bin_c(i) = (shared_cand.nodeZone1 .== i) + (shared_cand.nodeZone2 .== i) # A binary vector through which we can check if the shared candidate lines belong to zone i
bin_e(i) = (shared_ex.nodeZone1 .== i) + (shared_ex.nodeZone2 .== i) # A binary vector through which we can check if the shared existing lines belong to zone i
scen_weight(s) = scen_prob.scen_weight[scen_prob.scenario .== s,:]

scen_weight (generic function with 1 method)

#### Constructing the optimization model with a specified solver

In [6]:
Mod3 = Model(GLPK.Optimizer) #use different solver e.g. IPOPT, GLPK

A JuMP Model
Feasibility problem with:
Variables: 0
Model mode: AUTOMATIC
CachingOptimizer state: EMPTY_OPTIMIZER
Solver name: GLPK

In [7]:
zone_number = 3; #number of zones to consider

In [8]:
@variable(Mod3,0 <= gen_var_1[1:14,1:nrow(scen_prob)]) # generation at node 1
@variable(Mod3,0 <= gen_var_2[1:30,1:nrow(scen_prob)]) # generation at node 2
@variable(Mod3,0 <= gen_var_3[1:5,1:nrow(scen_prob)]) # generation at node 3

@variable(Mod3,shared_line_decision_var[1:nrow(shared_cand)], Bin) #Decision variable for shared candidate lines

@variable(Mod3,int_line_decision_var_1[1:nrow(int_c(1))], Bin) #Decision variable for internal candidate lines of zone 1 (Binary)
@variable(Mod3,int_line_decision_var_2[1:nrow(int_c(2))], Bin) #Decision variable for internal candidate lines of zone 2 (Binary)
@variable(Mod3,int_line_decision_var_3[1:nrow(int_c(3))], Bin) #Decision variable for internal candidate lines of zone 3 (Binary)

@variable(Mod3,shared_cand_flow[1:nrow(shared_cand),1:nrow(scen_prob)])  #Power flowing on shared candidate lines 

@variable(Mod3, int_cand_flow_1[1:nrow(int_c(1)),1:nrow(scen_prob)]) #Power flowing on internal candidate lines of zone 1 
@variable(Mod3, int_cand_flow_2[1:nrow(int_c(2)),1:nrow(scen_prob)]) #Power flowing on internal candidate lines of zone 2
@variable(Mod3, int_cand_flow_3[1:nrow(int_c(3)),1:nrow(scen_prob)]) #Power flowing on internal candidate lines of zone 3

@variable(Mod3, 0 <= node_voltage_phase_angle[i=1:nrow(zone_summary),j=1:zone_summary.Nodes_Total[i], 1:nrow(scen_prob)]<= 2*pi)

@variable(Mod3, shared_ex_flow[1:nrow(shared_ex),1:nrow(scen_prob)])  #Power flowing on shared existing shared lines

@variable(Mod3, int_ex_flow_1[1:nrow(int_e(1)),1:nrow(scen_prob)])  #Power flowing on internal existing lines of zone 1 
@variable(Mod3, int_ex_flow_2[1:nrow(int_e(2)),1:nrow(scen_prob)])  #Power flowing on internal existing lines of zone 2
@variable(Mod3, int_ex_flow_3[1:nrow(int_e(3)),1:nrow(scen_prob)])  #Power flowing on internal existing lines of zone 3


7×4 Array{VariableRef,2}:
 int_ex_flow_3[1,1]  int_ex_flow_3[1,2]  …  int_ex_flow_3[1,4]
 int_ex_flow_3[2,1]  int_ex_flow_3[2,2]     int_ex_flow_3[2,4]
 int_ex_flow_3[3,1]  int_ex_flow_3[3,2]     int_ex_flow_3[3,4]
 int_ex_flow_3[4,1]  int_ex_flow_3[4,2]     int_ex_flow_3[4,4]
 int_ex_flow_3[5,1]  int_ex_flow_3[5,2]     int_ex_flow_3[5,4]
 int_ex_flow_3[6,1]  int_ex_flow_3[6,2]  …  int_ex_flow_3[6,4]
 int_ex_flow_3[7,1]  int_ex_flow_3[7,2]     int_ex_flow_3[7,4]

In [9]:
@expression(Mod3,total_cost,0)

0

In [10]:
@expression(Mod3, gen_costNS1[n=1:14,s=1:nrow(scen_prob)], (gen_var_1[n,s] .* sum((g(1).gNodeID .== n) .* MC(1))))
@expression(Mod3, gen_cost_prob_weightedS1[s=1:nrow(scen_prob)], (scen_weight(s)[1]).*sum(gen_costNS1[n,s] for n in 1:14))

4-element Array{GenericAffExpr{Float64,VariableRef},1}:
 8.57573483 gen_var_1[1,1] + 13.75 gen_var_1[2,1]
 8.57573483 gen_var_1[1,2] + 13.75 gen_var_1[2,2]
 8.57573483 gen_var_1[1,3] + 13.75 gen_var_1[2,3]
 8.57573483 gen_var_1[1,4] + 13.75 gen_var_1[2,4]

In [11]:
@expression(Mod3, gen_cost_expected1, sum(gen_cost_prob_weightedS1[s] for s in 1:nrow(scen_prob)))
Mod3[:total_cost] += gen_cost_expected1

8.57573483 gen_var_1[1,1] + 13.75 gen_var_1[2,1] + 8.57573483 gen_var_1[1,2] + 13.75 gen_var_1[2,2] + 8.57573483 gen_var_1[1,3] + 13.75 gen_var_1[2,3] + 8.57573483 gen_var_1[1,4] + 13.75 gen_var_1[2,4]

In [12]:
@expression(Mod3, gen_costNS2[n=1:30,s=1:nrow(scen_prob)], (gen_var_2[n,s] .* sum((g(2).gNodeID .== n) .* MC(2))))

30×4 Array{GenericAffExpr{Float64,VariableRef},2}:
 3.6 gen_var_2[1,1]     3.6 gen_var_2[1,2]     …  3.6 gen_var_2[1,4]
 3.15 gen_var_2[2,1]    3.15 gen_var_2[2,2]       3.15 gen_var_2[2,4]
 0                      0                         0
 0                      0                         0
 0                      0                         0
 0                      0                      …  0
 0                      0                         0
 0                      0                         0
 0                      0                         0
 0                      0                         0
 0                      0                      …  0
 0                      0                         0
 3.5 gen_var_2[13,1]    3.5 gen_var_2[13,2]       3.5 gen_var_2[13,4]
 ⋮                                             ⋱  
 0                      0                         0
 0                      0                         0
 0                      0                      …  0
 3.667 gen_va

In [13]:
@expression(Mod3, gen_cost_prob_weightedS2[s=1:nrow(scen_prob)], (scen_weight(s)[1]).*sum(gen_costNS2[n,s] for n in 1:30))

4-element Array{GenericAffExpr{Float64,VariableRef},1}:
 0.9 gen_var_2[1,1] + 0.7875 gen_var_2[2,1] + 0.875 gen_var_2[13,1] + 0.91675 gen_var_2[22,1] + 0.9375 gen_var_2[23,1] + 1.09375 gen_var_2[27,1]
 0.9 gen_var_2[1,2] + 0.7875 gen_var_2[2,2] + 0.875 gen_var_2[13,2] + 0.91675 gen_var_2[22,2] + 0.9375 gen_var_2[23,2] + 1.09375 gen_var_2[27,2]
 0.9 gen_var_2[1,3] + 0.7875 gen_var_2[2,3] + 0.875 gen_var_2[13,3] + 0.91675 gen_var_2[22,3] + 0.9375 gen_var_2[23,3] + 1.09375 gen_var_2[27,3]
 0.9 gen_var_2[1,4] + 0.7875 gen_var_2[2,4] + 0.875 gen_var_2[13,4] + 0.91675 gen_var_2[22,4] + 0.9375 gen_var_2[23,4] + 1.09375 gen_var_2[27,4]

In [14]:
@expression(Mod3, gen_cost_expected2, sum(gen_cost_prob_weightedS2[s] for s in 1:nrow(scen_prob)))

0.9 gen_var_2[1,1] + 0.7875 gen_var_2[2,1] + 0.875 gen_var_2[13,1] + 0.91675 gen_var_2[22,1] + 0.9375 gen_var_2[23,1] + 1.09375 gen_var_2[27,1] + 0.9 gen_var_2[1,2] + 0.7875 gen_var_2[2,2] + 0.875 gen_var_2[13,2] + 0.91675 gen_var_2[22,2] + 0.9375 gen_var_2[23,2] + 1.09375 gen_var_2[27,2] + 0.9 gen_var_2[1,3] + 0.7875 gen_var_2[2,3] + 0.875 gen_var_2[13,3] + 0.91675 gen_var_2[22,3] + 0.9375 gen_var_2[23,3] + 1.09375 gen_var_2[27,3] + 0.9 gen_var_2[1,4] + 0.7875 gen_var_2[2,4] + 0.875 gen_var_2[13,4] + 0.91675 gen_var_2[22,4] + 0.9375 gen_var_2[23,4] + 1.09375 gen_var_2[27,4]

In [15]:
Mod3[:total_cost]+=gen_cost_expected2

8.57573483 gen_var_1[1,1] + 13.75 gen_var_1[2,1] + 8.57573483 gen_var_1[1,2] + 13.75 gen_var_1[2,2] + 8.57573483 gen_var_1[1,3] + 13.75 gen_var_1[2,3] + 8.57573483 gen_var_1[1,4] + 13.75 gen_var_1[2,4] + 0.9 gen_var_2[1,1] + 0.7875 gen_var_2[2,1] + 0.875 gen_var_2[13,1] + 0.91675 gen_var_2[22,1] + 0.9375 gen_var_2[23,1] + 1.09375 gen_var_2[27,1] + 0.9 gen_var_2[1,2] + 0.7875 gen_var_2[2,2] + 0.875 gen_var_2[13,2] + 0.91675 gen_var_2[22,2] + 0.9375 gen_var_2[23,2] + 1.09375 gen_var_2[27,2] + 0.9 gen_var_2[1,3] + 0.7875 gen_var_2[2,3] + 0.875 gen_var_2[13,3] + 0.91675 gen_var_2[22,3] + 0.9375 gen_var_2[23,3] + 1.09375 gen_var_2[27,3] + 0.9 gen_var_2[1,4] + 0.7875 gen_var_2[2,4] + 0.875 gen_var_2[13,4] + 0.91675 gen_var_2[22,4] + 0.9375 gen_var_2[23,4] + 1.09375 gen_var_2[27,4]

In [16]:
@expression(Mod3, gen_costNS3[n=1:5,s=1:nrow(scen_prob)], (gen_var_3[n,s] .* sum((g(3).gNodeID .== n) .* MC(3))))
@expression(Mod3, gen_cost_prob_weightedS3[s=1:nrow(scen_prob)], (scen_weight(s)[1]).*sum(gen_costNS3[n,s] for n in 1:5))

4-element Array{GenericAffExpr{Float64,VariableRef},1}:
 4.07573483 gen_var_3[1,1] + 9.25 gen_var_3[2,1]
 4.07573483 gen_var_3[1,2] + 9.25 gen_var_3[2,2]
 4.07573483 gen_var_3[1,3] + 9.25 gen_var_3[2,3]
 4.07573483 gen_var_3[1,4] + 9.25 gen_var_3[2,4]

In [17]:
@expression(Mod3, gen_cost_expected3, sum(gen_cost_prob_weightedS3[s] for s in 1:nrow(scen_prob)))

4.07573483 gen_var_3[1,1] + 9.25 gen_var_3[2,1] + 4.07573483 gen_var_3[1,2] + 9.25 gen_var_3[2,2] + 4.07573483 gen_var_3[1,3] + 9.25 gen_var_3[2,3] + 4.07573483 gen_var_3[1,4] + 9.25 gen_var_3[2,4]

In [18]:
Mod3[:total_cost]+=gen_cost_expected3

8.57573483 gen_var_1[1,1] + 13.75 gen_var_1[2,1] + 8.57573483 gen_var_1[1,2] + 13.75 gen_var_1[2,2] + 8.57573483 gen_var_1[1,3] + 13.75 gen_var_1[2,3] + 8.57573483 gen_var_1[1,4] + 13.75 gen_var_1[2,4] + 0.9 gen_var_2[1,1] + 0.7875 gen_var_2[2,1] + 0.875 gen_var_2[13,1] + 0.91675 gen_var_2[22,1] + 0.9375 gen_var_2[23,1] + 1.09375 gen_var_2[27,1] + 0.9 gen_var_2[1,2] + 0.7875 gen_var_2[2,2] + 0.875 gen_var_2[13,2] + 0.91675 gen_var_2[22,2] + 0.9375 gen_var_2[23,2] + 1.09375 gen_var_2[27,2] + 0.9 gen_var_2[1,3] + 0.7875 gen_var_2[2,3] + 0.875 gen_var_2[13,3] + 0.91675 gen_var_2[22,3] + 0.9375 gen_var_2[23,3] + 1.09375 gen_var_2[27,3] + 0.9 gen_var_2[1,4] + 0.7875 gen_var_2[2,4] + 0.875 gen_var_2[13,4] + 0.91675 gen_var_2[22,4] + 0.9375 gen_var_2[23,4] + 1.09375 gen_var_2[27,4] + 4.07573483 gen_var_3[1,1] + 9.25 gen_var_3[2,1] + 4.07573483 gen_var_3[1,2] + 9.25 gen_var_3[2,2] + 4.07573483 gen_var_3[1,3] + 9.25 gen_var_3[2,3] + 4.07573483 gen_var_3[1,4] + 9.25 gen_var_3[2,4]

In [19]:
@expression(Mod3, investment_cost , (sum(shared_line_decision_var[c] .* shared_cand.costPerCap[c] .* shared_cand.interestRate[c] 
            .*((1 + shared_cand.interestRate[c]) .^ shared_cand.lifeTime[c]) ./ (((1 + shared_cand.interestRate[c]) .^ shared_cand.lifeTime[c])-1) for c in 1:nrow(shared_cand))
            .+ sum(int_line_decision_var_1[c] .* int_c(1).costPerCap[c] .* int_c(1).interestRate[c] 
            .*((1 + int_c(1).interestRate[c]) .^ int_c(1).lifeTime[c]) ./ (((1 + int_c(1).interestRate[c]) .^ int_c(1).lifeTime[c])-1) for c in 1:nrow(int_c(1)))
            .+ sum(int_line_decision_var_2[c] .* int_c(2).costPerCap[c] .* int_c(2).interestRate[c] 
            .*((1 + int_c(2).interestRate[c]) .^ int_c(2).lifeTime[c]) ./ (((1 + int_c(2).interestRate[c]) .^ int_c(2).lifeTime[c])-1) for c in 1:nrow(int_c(2)))
            .+ sum(int_line_decision_var_3[c] .* int_c(3).costPerCap[c] .* int_c(3).interestRate[c] 
            .*((1 + int_c(3).interestRate[c]) .^ int_c(3).lifeTime[c]) ./ (((1 + int_c(3).interestRate[c]) .^ int_c(3).lifeTime[c])-1) for c in 1:nrow(int_c(3)))))
Mod3[:total_cost]+=investment_cost

8.57573483 gen_var_1[1,1] + 13.75 gen_var_1[2,1] + 8.57573483 gen_var_1[1,2] + 13.75 gen_var_1[2,2] + 8.57573483 gen_var_1[1,3] + 13.75 gen_var_1[2,3] + 8.57573483 gen_var_1[1,4] + 13.75 gen_var_1[2,4] + 0.9 gen_var_2[1,1] + 0.7875 gen_var_2[2,1] + 0.875 gen_var_2[13,1] + 0.91675 gen_var_2[22,1] + 0.9375 gen_var_2[23,1] + 1.09375 gen_var_2[27,1] + 0.9 gen_var_2[1,2] + 0.7875 gen_var_2[2,2] + 0.875 gen_var_2[13,2] + 0.91675 gen_var_2[22,2] + 0.9375 gen_var_2[23,2] + 1.09375 gen_var_2[27,2] + 0.9 gen_var_2[1,3] + 0.7875 gen_var_2[2,3] + 0.875 gen_var_2[13,3] + 0.91675 gen_var_2[22,3] + 0.9375 gen_var_2[23,3] + 1.09375 gen_var_2[27,3] + 0.9 gen_var_2[1,4] + 0.7875 gen_var_2[2,4] + 0.875 gen_var_2[13,4] + 0.91675 gen_var_2[22,4] + 0.9375 gen_var_2[23,4] + 1.09375 gen_var_2[27,4] + 4.07573483 gen_var_3[1,1] + 9.25 gen_var_3[2,1] + 4.07573483 gen_var_3[1,2] + 9.25 gen_var_3[2,2] + 4.07573483 gen_var_3[1,3] + 9.25 gen_var_3[2,3] + 4.07573483 gen_var_3[1,4] + 9.25 gen_var_3[2,4] + 0.2410176361

In [20]:
#The first zone
for n in 1:14
    for s in 1:nrow(scen_prob)
        # Power balance constraint for each node
        @constraint(Mod3, sum(g(1).gNodeID .== n) .* gen_var_1[n,s] .+ sum(l(1,s)[:,3].* (l(1,s).lNodeID .== n))./100 .==
        sum((shared_cand.tNodeID1 .== n) .*bin_c(1) .* shared_cand_flow[:,s]) .- sum((shared_cand.tNodeID2 .== n) .* bin_c(1) .* shared_cand_flow[:,s]) .+
        sum((shared_ex.tNodeID1 .== n) .* bin_e(1) .* shared_ex_flow[:,s]) .- sum((shared_ex.tNodeID2 .== n) .* bin_e(1) .* shared_ex_flow[:,s]) .+
        sum((int_c(1).tNodeID1 .== n) .* int_cand_flow_1[:,s]) .- sum((int_c(1).tNodeID2 .== n) .* int_cand_flow_1[:,s]) .+
        sum((int_e(1).tNodeID1 .== n) .* int_ex_flow_1[:,s]) .- sum((int_e(1).tNodeID2 .== n) .* int_ex_flow_1[:,s]))
        #Lower limit for generation of each node
        @constraint(Mod3, sum(g(1).gNodeID .== n) .* gen_var_1[n,s] .<= sum((g(1).gNodeID .== n) .* g(1).PgMax)./100)
        #Upper limit for generation of each node
        @constraint(Mod3, sum((g(1).gNodeID .== n) .* g(1).PgMin)./100 .<= sum(g(1).gNodeID .== n) .* gen_var_1[n,s])
    end
end

In [21]:
#The second zone
for n in 1:30
    for s in 1:nrow(scen_prob)
        # Power balance constraint for each node
        @constraint(Mod3, sum(g(2).gNodeID .== n) .* gen_var_2[n,s] .+ sum(l(2,s)[:,3] .* (l(2,s).lNodeID .== n))./100 .==
        sum((shared_cand.tNodeID1 .== n) .*bin_c(2) .* shared_cand_flow[:,s]) .- sum((shared_cand.tNodeID2 .== n) .* bin_c(2) .* shared_cand_flow[:,s]) .+
        sum((shared_ex.tNodeID1 .== n) .* bin_e(2) .* shared_ex_flow[:,s]) .- sum((shared_ex.tNodeID2 .== n) .* bin_e(2) .* shared_ex_flow[:,s]) .+
        sum((int_c(2).tNodeID1 .== n) .* int_cand_flow_2[:,s]) .- sum((int_c(2).tNodeID2 .== n) .* int_cand_flow_2[:,s]) .+
        sum((int_e(2).tNodeID1 .== n) .* int_ex_flow_2[:,s]) .- sum((int_e(2).tNodeID2 .== n) .* int_ex_flow_2[:,s]))
        #Lower limit for generation of each node
        @constraint(Mod3, sum(g(2).gNodeID .== n) .* gen_var_2[n,s] .<= sum((g(2).gNodeID .== n) .* g(2).PgMax)./100)
        #Upper limit for generation of each node
        @constraint(Mod3, sum((g(2).gNodeID .== n) .* g(2).PgMin)./100 .<= sum(g(2).gNodeID .== n) .* gen_var_2[n,s])
    end
end

In [22]:
#The third zone
for n in 1:5
    for s in 1:nrow(scen_prob)
        # Power balance constraint for each node
        @constraint(Mod3, sum(g(3).gNodeID .== n) .* gen_var_3[n,s] .+ sum(l(3,s)[:,3] .* (l(3,s).lNodeID .== n))./100 .==
        sum((shared_cand.tNodeID1 .== n) .*bin_c(3) .* shared_cand_flow[:,s]) .- sum((shared_cand.tNodeID2 .== n) .* bin_c(3) .* shared_cand_flow[:,s]) .+
        sum((shared_ex.tNodeID1 .== n) .* bin_e(3) .* shared_ex_flow[:,s]) .- sum((shared_ex.tNodeID2 .== n) .* bin_e(3) .* shared_ex_flow[:,s]) .+
        sum((int_c(3).tNodeID1 .== n) .* int_cand_flow_3[:,s]) .- sum((int_c(3).tNodeID2 .== n) .* int_cand_flow_3[:,s]) .+
        sum((int_e(3).tNodeID1 .== n) .* int_ex_flow_3[:,s]) .- sum((int_e(3).tNodeID2 .== n) .* int_ex_flow_3[:,s]))
        #Lower limit for generation of each node
        @constraint(Mod3, sum(g(3).gNodeID .== n) .* gen_var_3[n,s] .<= sum((g(3).gNodeID .== n) .* g(3).PgMax)./100)
        #Upper limit for generation of each node
        @constraint(Mod3, sum((g(3).gNodeID .== n) .* g(3).PgMin)./100 .<= sum(g(3).gNodeID .== n) .* gen_var_3[n,s])
    end
end

In [23]:
#Shared candidate lines
M = 100 # this should be changed/translated to PU value...
for c in 1:nrow(shared_cand)
    for s in 1:nrow(scen_prob)
        @constraint(Mod3,-M .* (1 .- shared_line_decision_var[c]) .<= shared_cand_flow[c,s] .- ((1 ./ shared_cand.reacT[c]) .* (node_voltage_phase_angle[shared_cand.nodeZone1[c],shared_cand.tNodeID1[c],s] .- node_voltage_phase_angle[shared_cand.nodeZone2[c],shared_cand.tNodeID2[c],s])))
        @constraint(Mod3, shared_cand_flow[c,s] .- ((1 ./ shared_cand.reacT[c]) .* (node_voltage_phase_angle[shared_cand.nodeZone1[c],shared_cand.tNodeID1[c],s] .- node_voltage_phase_angle[shared_cand.nodeZone2[c],shared_cand.tNodeID2[c],s])) .<= M .* (1 .- shared_line_decision_var[c]))
        #limiting the upper bound of power flow flowing within candidate shared lines
        @constraint(Mod3, shared_cand_flow[c,s] .<= shared_line_decision_var[c] .*shared_cand.ptMax[c]./100)
        #Limiting the lower bound of power flowing within the candidate shared lines
        @constraint(Mod3, -(shared_cand.ptMax[c]./100) .* shared_line_decision_var[c] .<= shared_cand_flow[c,s])
    end
end

In [24]:
#Shared existing lines
for h in 1:nrow(shared_ex)
    for s in 1:nrow(scen_prob)
        @constraint(Mod3, shared_ex_flow[h,s] .== (1 ./ shared_ex.reacT[h]) .* (node_voltage_phase_angle[shared_ex.nodeZone1[h],shared_ex.tNodeID1[h],s] .- node_voltage_phase_angle[shared_ex.nodeZone2[h],shared_ex.tNodeID2[h],s]))
        @constraint(Mod3, shared_ex_flow[h,s] .<= shared_ex. ptMax[h]./100)
        @constraint(Mod3, -shared_ex.ptMax[h]./100 .<= shared_ex_flow[h,s])
    end
end

In [25]:
#Zone 1 internal candidate lines
for c in 1:nrow(int_c(1))
    for s in 1:nrow(scen_prob)
        @constraint(Mod3,-M .* (1 .- int_line_decision_var_1[c]) .<= int_cand_flow_1[c,s] .- ((1 ./ int_c(1).reacT[c]) .* (node_voltage_phase_angle[int_c(1).zoneNum[c],int_c(1).tNodeID1[c],s] .- node_voltage_phase_angle[int_c(1).zoneNum[c],int_c(1).tNodeID2[c],s])))
        @constraint(Mod3, int_cand_flow_1[c,s] .- ((1 ./ int_c(1).reacT[c]) .* (node_voltage_phase_angle[int_c(1).zoneNum[c],int_c(1).tNodeID1[c],s] .- node_voltage_phase_angle[int_c(1).zoneNum[c],int_c(1).tNodeID2[c],s])) .<= M .* (1 .- int_line_decision_var_1[c]))
        @constraint(Mod3, int_cand_flow_1[c,s] .<= int_line_decision_var_1[c] .* int_c(1).ptMax[c]./100)
        @constraint(Mod3, -(int_c(1).ptMax[c]./100) .*int_line_decision_var_1[c] .<= int_cand_flow_1[c,s])
    end
end

In [26]:
#Zone 2 internal candidate lines
for c in 1:nrow(int_c(2))
    for s in 1:nrow(scen_prob)
        @constraint(Mod3,-M .* (1 .- int_line_decision_var_2[c]) .<= int_cand_flow_2[c,s] .- ((1 ./ int_c(2).reacT[c]) .* (node_voltage_phase_angle[int_c(2).zoneNum[c],int_c(2).tNodeID1[c],s] .- node_voltage_phase_angle[int_c(2).zoneNum[c],int_c(2).tNodeID2[c],s])))
        @constraint(Mod3, int_cand_flow_2[c,s] .- ((1 ./ int_c(2).reacT[c]) .* (node_voltage_phase_angle[int_c(2).zoneNum[c],int_c(2).tNodeID1[c],s] .- node_voltage_phase_angle[int_c(2).zoneNum[c],int_c(2).tNodeID2[c],s])) .<= M .* (1 .- int_line_decision_var_2[c]))
        @constraint(Mod3, int_cand_flow_2[c,s] .<= int_line_decision_var_2[c] .* int_c(2).ptMax[c]./100)
        @constraint(Mod3, -(int_c(2).ptMax[c]./100) .*int_line_decision_var_2[c] .<= int_cand_flow_2[c,s])
    end
end

In [27]:
#Zone 3 internal candidate lines
for c in 1:nrow(int_c(3))
    for s in 1:nrow(scen_prob)
        @constraint(Mod3,-M .* (1 .- int_line_decision_var_3[c]) .<= int_cand_flow_3[c,s] .- ((1 ./ int_c(3).reacT[c]) .* (node_voltage_phase_angle[int_c(3).zoneNum[c],int_c(3).tNodeID1[c],s] .- node_voltage_phase_angle[int_c(3).zoneNum[c],int_c(3).tNodeID2[c],s])))
        @constraint(Mod3, int_cand_flow_3[c,s] .- ((1 ./ int_c(3).reacT[c]) .* (node_voltage_phase_angle[int_c(3).zoneNum[c],int_c(3).tNodeID1[c],s] .- node_voltage_phase_angle[int_c(3).zoneNum[c],int_c(3).tNodeID2[c],s])) .<= M .* (1 .- int_line_decision_var_3[c]))
        @constraint(Mod3, int_cand_flow_3[c,s] .<= int_line_decision_var_3[c] .* int_c(3).ptMax[c]./100)
        @constraint(Mod3, -(int_c(3).ptMax[c]./100) .*int_line_decision_var_3[c] .<= int_cand_flow_3[c,s])
    end
end

In [28]:
#Zone 1 internal existing lines
for h in 1:nrow(int_e(1))
    for s in 1:nrow(scen_prob)
        @constraint(Mod3, int_ex_flow_1[h,s] .== (1 ./ int_e(1).reacT[h]) .* (node_voltage_phase_angle[int_e(1).zoneNum[h],int_e(1).tNodeID1[h],s] .- node_voltage_phase_angle[int_e(1).zoneNum[h],int_e(1).tNodeID2[h],s]))
        @constraint(Mod3, int_ex_flow_1[h,s] .<= int_e(1).ptMax[h]./100)
        @constraint(Mod3, -int_e(1).ptMax[h]./100 .<= int_ex_flow_1[h,s])
    end
end

In [29]:
#Zone 2 internal existing lines
for h in 1:nrow(int_e(2))
    for s in 1:nrow(scen_prob)
        @constraint(Mod3, int_ex_flow_2[h,s] .== (1 ./ int_e(2).reacT[h]) .* (node_voltage_phase_angle[int_e(2).zoneNum[h],int_e(2).tNodeID1[h],s] .- node_voltage_phase_angle[int_e(2).zoneNum[h],int_e(2).tNodeID2[h],s]))
        @constraint(Mod3, int_ex_flow_2[h,s] .<= int_e(2).ptMax[h]./100)
        @constraint(Mod3, -int_e(2).ptMax[h]./100 .<= int_ex_flow_2[h,s])
    end
end

In [30]:
#Zone 3 internal existing lines
for h in 1:nrow(int_e(3))
    for s in 1:nrow(scen_prob)
        @constraint(Mod3, int_ex_flow_3[h,s] .== (1 ./ int_e(3).reacT[h]) .* (node_voltage_phase_angle[int_e(3).zoneNum[h],int_e(3).tNodeID1[h],s] .- node_voltage_phase_angle[int_e(3).zoneNum[h],int_e(3).tNodeID2[h],s]))
        @constraint(Mod3, int_ex_flow_3[h,s] .<= int_e(3).ptMax[h]./100)
        @constraint(Mod3, -int_e(3).ptMax[h]./100 .<= int_ex_flow_3[h,s])
    end
end

In [31]:
@objective(Mod3, Min, Mod3[:total_cost])
optimize!(Mod3)

In [32]:
termination_status(Mod3)

OPTIMAL::TerminationStatusCode = 1

In [33]:
if termination_status(Mod3) == MOI.OPTIMAL
    shared_line_decision = value.(shared_line_decision_var)
    shared_cand_power = (value.(shared_cand_flow)) .* 100
    node_voltage_phase_angle = value.(node_voltage_phase_angle)
    shared_ex_power = (value.(shared_ex_flow)) .* 100
    int_line_decision_1 = value.(int_line_decision_var_1)
    int_line_decision_2 = value.(int_line_decision_var_2)
    int_line_decision_3 = value.(int_line_decision_var_3)
    int_line_flow_1 = (value.(int_cand_flow_1)) .* 100
    int_line_flow_2 = (value.(int_cand_flow_2)) .* 100
    int_line_flow_3 = (value.(int_cand_flow_3)) .* 100
    generation_1 = (value.(gen_var_1)) .* 100
    generation_2 = (value.(gen_var_2)) .* 100
    generation_3 = (value.(gen_var_3)) .* 100
    internal_existing_line_flow_1 = (value.(int_ex_flow_1)) .* 100
    internal_existing_line_flow_2 = (value.(int_ex_flow_2)) .* 100
    internal_existing_line_flow_3 = (value.(int_ex_flow_3)) .* 100
    obj_value = objective_value(Mod3)
end

27.2982547052472

In [34]:
shared_line_decision

6-element Array{Float64,1}:
 0.0
 0.0
 0.0
 1.0
 1.0
 0.0

In [35]:
shared_cand_power

6×4 Array{Float64,2}:
 -0.0     -0.0     -0.0     -0.0
 -0.0     -0.0     -0.0      0.0
 -0.0     -0.0     -0.0     -0.0
 91.1514  84.4184  95.2955  43.7686
 92.1652  62.7053  85.5807  35.2138
  0.0     -0.0     -0.0     -0.0

In [36]:
node_voltage_phase_angle

JuMP.Containers.SparseAxisArray{Float64,3,Tuple{Int64,Int64,Int64}} with 196 entries:
  [1, 1, 2 ]  =  0.0466216
  [1, 11, 2]  =  0.151361
  [1, 12, 4]  =  0.124647
  [1, 14, 2]  =  0.178808
  [1, 2, 1 ]  =  0.095336
  [1, 4, 2 ]  =  0.0464514
  [1, 5, 3 ]  =  0.0881079
  [1, 8, 2 ]  =  0.118346
  [2, 14, 4]  =  0.260723
  [2, 22, 1]  =  0.425613
  [2, 23, 2]  =  0.365958
              ⋮
  [2, 24, 3]  =  0.417782
  [2, 26, 1]  =  0.475313
  [2, 27, 1]  =  0.560085
  [2, 3, 2 ]  =  0.418741
  [2, 30, 2]  =  0.339507
  [2, 4, 4 ]  =  0.232144
  [2, 9, 1 ]  =  0.404866
  [2, 9, 4 ]  =  0.172411
  [3, 4, 2 ]  =  0.262369
  [3, 4, 4 ]  =  0.145331

In [37]:
shared_ex_power

8×4 Array{Float64,2}:
  49.786    40.3714   47.8922   20.9814
 -64.5571  -50.7171  -62.2156  -27.6916
 100.0     100.0     100.0      58.5871
  38.6976   35.4737   38.0971   20.0223
 -74.7264  -52.6075  -61.0791  -30.5716
  60.388    35.8198   53.1297   24.5465
 100.0      65.2039   97.0733   38.3456
 -44.7579  -36.5825  -46.1188  -12.0613

In [38]:
int_line_decision_1

10-element Array{Float64,1}:
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 1.0

In [39]:
int_line_decision_2

18-element Array{Float64,1}:
 1.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 1.0
 1.0
 0.0
 0.0
 1.0
 0.0
 1.0
 0.0
 0.0

In [40]:
int_line_decision_3

2-element Array{Float64,1}:
 0.0
 0.0

In [41]:
obj_value

27.2982547052472

In [42]:
generation_1

14×4 Array{Float64,2}:
 118.063  4.71029  51.0623  0.0
   0.0    0.0       0.0     0.0
   0.0    0.0       0.0     0.0
   0.0    0.0       0.0     0.0
   0.0    0.0       0.0     0.0
   0.0    0.0       0.0     0.0
   0.0    0.0       0.0     0.0
   0.0    0.0       0.0     0.0
   0.0    0.0       0.0     0.0
   0.0    0.0       0.0     0.0
   0.0    0.0       0.0     0.0
   0.0    0.0       0.0     0.0
   0.0    0.0       0.0     0.0
   0.0    0.0       0.0     0.0

In [43]:
show(stdout, "text/plain", generation_2)

30×4 Array{Float64,2}:
  0.0   0.0   0.0   0.0
 14.2   4.6  51.9  47.3
  0.0   0.0   0.0   0.0
  0.0   0.0   0.0   0.0
  0.0   0.0   0.0   0.0
  0.0   0.0   0.0   0.0
  0.0   0.0   0.0   0.0
  0.0   0.0   0.0   0.0
  0.0   0.0   0.0   0.0
  0.0   0.0   0.0   0.0
  0.0   0.0   0.0   0.0
  0.0   0.0   0.0   0.0
 40.0  40.0  40.0   0.0
  0.0   0.0   0.0   0.0
  0.0   0.0   0.0   0.0
  0.0   0.0   0.0   0.0
  0.0   0.0   0.0   0.0
  0.0   0.0   0.0   0.0
  0.0   0.0   0.0   0.0
  0.0   0.0   0.0   0.0
  0.0   0.0   0.0   0.0
 50.0  50.0  50.0   0.0
 30.0   0.0   0.0   0.0
  0.0   0.0   0.0   0.0
  0.0   0.0   0.0   0.0
  0.0   0.0   0.0   0.0
 55.0   0.0   0.0   0.0
  0.0   0.0   0.0   0.0
  0.0   0.0   0.0   0.0
  0.0   0.0   0.0   0.0

In [44]:
generation_3

5×4 Array{Float64,2}:
 42.4154  0.0  0.0  0.0
  0.0     0.0  0.0  0.0
  0.0     0.0  0.0  0.0
  0.0     0.0  0.0  0.0
  0.0     0.0  0.0  0.0

In [45]:
int_line_flow_1

10×4 Array{Float64,2}:
 -0.0     -0.0      0.0      0.0
 -0.0     -0.0     -0.0     -0.0
 -0.0     -0.0     -0.0      0.0
 -0.0     -0.0     -0.0      0.0
  0.0      0.0      0.0     -0.0
 -0.0      0.0     -0.0      0.0
 -0.0      0.0      0.0      0.0
 -0.0      0.0      0.0      0.0
  0.0     -0.0     -0.0      0.0
 39.7704  51.6737  49.9709  28.3685

In [46]:
int_line_flow_2

18×4 Array{Float64,2}:
 -65.6706  -51.2704   -78.7672  -38.2372
  -0.0      -0.0       -0.0       0.0
  -0.0       0.0       -0.0      -0.0
   0.0       0.0        0.0       0.0
  -0.0       0.0        0.0      -0.0
  -0.0       0.0        0.0      -0.0
  -0.0      -0.0       -0.0      -0.0
   0.0       0.0        0.0       0.0
   0.0       0.0        0.0       0.0
 -41.4845  -36.3283   -59.181   -30.0504
 -46.2175  -40.5114   -46.5053  -20.1975
  -0.0      -0.0       -0.0       0.0
  -0.0      -0.0       -0.0      -0.0
 -18.7456  -16.0081   -18.6725   -8.34003
  -0.0      -0.0       -0.0      -0.0
 -10.6871   -8.05562  -10.3934   -5.28197
  -0.0      -0.0       -0.0      -0.0
  -0.0      -0.0        0.0       0.0

In [47]:
int_line_flow_3

2×4 Array{Float64,2}:
 -0.0  -0.0  -0.0  -0.0
  0.0   0.0   0.0  -0.0

In [48]:
internal_existing_line_flow_1

20×4 Array{Float64,2}:
  70.8603    12.077      39.23         3.58472
  12.2464    -8.30051     0.724104    -5.78779
  48.1568    19.9402     33.5963       9.5859
   9.40803   -3.95628     1.77951     -2.7822
  -8.4045   -14.757     -12.4208      -8.64398
 -46.0432   -27.1598    -37.0537     -13.9641
 -74.0964   -44.3688    -58.739      -24.0432
 -37.4154   -34.3796    -38.0961     -18.3159
 -11.6211   -11.7414    -12.3862      -6.35947
 -39.1569   -35.7526    -38.0386     -20.3526
   5.88581    1.94982     2.54521      1.46572
 -35.9134   -26.5318    -30.1645     -15.3814
 -20.3293   -16.7706    -18.8194      -9.23701
   0.0        0.0         0.0          0.0
  12.3706     5.99176     9.79618      2.6654
 -57.9429   -46.417     -55.3858     -26.0323
 -35.3647   -24.7998    -31.5448     -12.7283
  -2.38581   -0.199816    0.0797939   -0.590724
  32.7131    23.0257     26.3397      13.6652
  -1.11626   -0.494874   -2.60468      1.05321

In [49]:
internal_existing_line_flow_2

41×4 Array{Float64,2}:
  -9.89323    -2.56512  -13.4613   -11.754
   9.89323     2.56512   13.4613    11.754
  24.0849     11.5686    29.8659    20.1352
 -45.3943    -33.1284   -42.2114   -14.068
  22.8197     12.2362    28.4578    18.5499
  52.2512     37.9476    57.7801    23.2153
  -3.87524     3.56954   -4.12051   -5.48645
  -0.894129  -11.6225    -4.19331    5.568
  54.6057     32.0755    54.8619    24.2776
 -81.5917    -61.0469   -85.6542   -39.5589
  24.288      14.7554    21.8657     7.40627
  59.2747     40.8384    56.7942    29.8099
  19.0153     11.7295    18.1179    10.526
   ⋮                               
  57.2608     49.9699    57.4996    25.6702
 -74.4309    -63.5617   -74.1407   -33.1148
 -16.5362    -12.7167   -11.3952     0.317542
 -13.416      -3.28325   -8.44374   -9.34902
  10.2733      6.09009    8.71305    5.35874
 -11.8427     -1.54317   -6.25569   -6.16528
   3.5         1.75       2.625      0.875
 -34.2462     -2.78466   -2.82934   -1.53636
  -7.75383     

In [50]:
internal_existing_line_flow_3

7×4 Array{Float64,2}:
 -37.6226   -47.2224   -57.3219   -28.799
   5.31155   -5.38517   -3.75725   -1.77256
  19.6229     8.56056   14.0976     7.23625
  23.7383    11.4369    17.8825     8.28208
 -40.5959   -41.4      -51.1723   -24.7709
  24.6924    17.2578    22.7092     6.27495
 -38.1017   -29.2776   -38.998    -18.597